<a href="https://colab.research.google.com/github/keshav-b/ML-DL-stuff/blob/master/GANs/GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import tqdm.notebook as tq

In [2]:
transforms = transforms.Compose(
                                  [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),]
                               )

In [6]:
batch_size = 32
dataset = datasets.FashionMNIST(root="dataset/", transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Simple GAN

In [7]:
class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.LeakyReLU(0.01),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x)

In [8]:
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.01),
            nn.Linear(256, img_dim),
            nn.Tanh(),  # normalize inputs to [-1, 1] so make outputs [-1, 1]
        )

    def forward(self, x):
        return self.gen(x)

In [9]:
# Hyperparameters etc.
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64
image_dim = 28 * 28 * 1  # 784
num_epochs = 5

In [10]:
disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)

In [11]:
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()

In [12]:
for epoch in range(num_epochs):
    for x, _ in tq.tqdm(loader):
        x = x.view(-1, 784).to(device)
        batch_size = x.shape[0]

        # DISC TRAIN: obj = max log(D(x)) + log(1-D(G(z)))
        #             obj = max log(D_x) + log(1-D_G_z)

        D_x = disc(x).view(-1)

        z = torch.randn(batch_size, z_dim).to(device) # noise
        G_z = gen(z)
        D_G_z = disc(G_z.detach()).view(-1) # flatten | detach: re-use G_z for GEN

        lossD = criterion(D_x, torch.ones_like(D_x)) + criterion(D_G_z, torch.zeros_like(D_G_z))
        lossD  = lossD / 2   # ????

        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        # GEN TRAIN: obj = min  log(1-D(G(z))) = max log(D(G(z)))
        #             obj = min  log(1-D_G_z_) = max log(1-D_G_z_)

        D_G_z_ = disc(G_z).view(-1) # flatten

        lossG = criterion(D_G_z_, torch.ones_like(D_G_z_))

        gen.zero_grad()
        lossG.backward()
        opt_gen.step()


    print(f"Epoch [{epoch}/{num_epochs}] \t LossD: {lossD:.4f} \t LossG: {lossG:.4f}")


Epoch [0/5] 	 LossD: 0.2038 	 LossG: 1.7877



Epoch [1/5] 	 LossD: 0.2965 	 LossG: 2.1221



Epoch [2/5] 	 LossD: 0.3500 	 LossG: 1.9642



Epoch [3/5] 	 LossD: 0.5073 	 LossG: 1.3444



Epoch [4/5] 	 LossD: 0.4392 	 LossG: 1.4552


# DCGAN


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.LeakyReLU(0.01),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x)

In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.01),
            nn.Linear(256, img_dim),
            nn.Tanh(),  # normalize inputs to [-1, 1] so make outputs [-1, 1]
        )

    def forward(self, x):
        return self.gen(x)